In [1]:
# Base Package
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import gc
import glob

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# DNN Package
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import timm

In [3]:
# My Package
from shopee_dataset import *
from my_nn_trainer import *
from metric import *

In [4]:
INPUT_DIR = os.path.join("..","..","input","shopee-product-matching")

df = pd.read_csv(os.path.join(INPUT_DIR,"train.csv"))
df['filepath'] = df['image'].apply(lambda x: os.path.join(INPUT_DIR, 'train_images', x))


from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['label_group'] = encoder.fit_transform(df['label_group'])

TRAIN_BATCH_SIZE = 5
train_loader = torch.utils.data.DataLoader(
                                            ShopeeDataset(csv=df, transforms=get_train_transforms()),
                                            batch_size=TRAIN_BATCH_SIZE,
                                            pin_memory=True,
                                            drop_last=True,
                                            #num_workers=4
                                           )

valid_loader = torch.utils.data.DataLoader(
                                            ShopeeDataset(csv=df, transforms=get_train_transforms()),
                                            batch_size=TRAIN_BATCH_SIZE,
                                            pin_memory=True,
                                            drop_last=True,
                                            #num_workers=4
                                           )

CLASS_NUM = len(df["label_group"].unique())

C:\Users\organ\anaconda3\lib\site-packages\albumentations\augmentations\transforms.py:1699: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [5]:
# CNN部分の前処理
cnn_model = timm.create_model(model_name='efficientnet_b0',pretrained=True)
cnn_model.global_pool = nn.Flatten()
cnn_model.classifier = nn.Identity()

#print(cnn_model)

In [6]:
metric_model = AdaCos(num_features=327680, num_classes=CLASS_NUM)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.0001)

In [7]:
run_training(cnn_model=cnn_model,
             metric_model=metric_model,
             optimizer=optimizer,
             scheduler=None,
             criterion=criterion,
             metric="accuracy",
             epoch_num=5,
             train_dataloader=train_loader,
             valid_dataloader=valid_loader,
             MODEL_NAME="test",
             device="cuda",
             verbose=-1)

RuntimeError: CUDA out of memory. Tried to allocate 13.45 GiB (GPU 0; 8.00 GiB total capacity; 15.52 MiB already allocated; 6.16 GiB free; 34.00 MiB reserved in total by PyTorch)